# Detrend with different start and end dates to 1979-2020
use the reduced datasets of 1979-2020 but only use 1979-2010 or 1989-2020


In [1]:
import numpy as np
import xarray as xr
import datetime
import scipy.signal as signal

import matplotlib.pyplot as plt

print(datetime.datetime.utcnow().strftime("%H:%M UTC %a %Y-%m-%d"))

00:22 UTC Fri 2022-04-15


In [2]:
data_path = '/glade/scratch/cwpowell/Synthetic_ensemble/'

month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 
               'August', 'September', 'October', 'November', 'December']

model_names  = ['CanESM2', 'CESM1', 'CSIRO_MK36', 'GFDL_CM3', 'GFDL_ESM2M', 'MPI_ESM1' ]
mem_len      = [50,        40,      30,           20,         30,           100        ]
model_starts = [1950,      1920,    1850,         1920,       1950,         1850       ]

# Models
**Start from reduced datasets made in https://github.com/chrisrwp/synthetic-ensemble/blob/main/SIC/Detrend_SIC_models.ipynb** <br>
No need to run dask workers, takes ~6 minutes to run, requires ~10 GB RAM

In [5]:
start_yr = 1979 #1979
end_yr = 2010 #2010

for model_i, model_name in enumerate(model_names):
    print(datetime.datetime.now(), model_name)
    
    #define grid chatachteristics
    if model_name in ['CESM1', 'MPI_ESM1']:
        lat_lon = ['j', 'i']
    elif  model_name == 'GFDL_CM3':
        lat_lon = ['rlat', 'rlon']
    else:
        lat_lon = ['lat', 'lon']
    
    for month_ in np.arange(1,13):
        print(datetime.datetime.now(), month_)
        ##############################################################################################
        #load the reduced datasets, each month for each model for 30N+ and 1979-2020
        month_data_ = xr.open_dataarray('/glade/campaign/univ/ucub0084/Synthetic_ensemble/SIC/Reduced_datasets/'\
                                        +'{}_reduced_1979-2020_{}.nc'.format(model_name, str(month_).zfill(2)))
        month_data_ = month_data_.sel(time=slice(str(start_yr),str(end_yr)))
        
        #change the time to whole numbers for ease of trend calculations
        month_data = month_data_.copy()
        month_data['time'] = np.arange(start_yr,end_yr+1)
        
        #GFDL ESM2M uses fraction not percentage for SIC, change to % for this model
        if model_name == 'GFDL_ESM2M': month_data = month_data * 100
        
        ##############################################################################################            
        #generate a matrix of year values for computing the trend with the trend coefficients
        yrs_ind = np.empty((len(month_data['time']), len(month_data['member']), 
                            len(month_data[lat_lon[0]]), len(month_data[lat_lon[1]])))

        for yr_i, yr in enumerate(np.arange(start_yr,end_yr+1)):
            yrs_ind[yr_i] = np.ones((len(month_data['member']), len(month_data[lat_lon[0]]), 
                                     len(month_data[lat_lon[1]]))) * yr

        yrs_ind = xr.DataArray(data = yrs_ind, coords = {'time':month_data['time'], 'member':month_data['member'], 
                               lat_lon[0]:month_data[lat_lon[0]], lat_lon[1]:month_data[lat_lon[1]]}, 
                               dims = ['time', 'member', lat_lon[0], lat_lon[1]])          
        
        #use ordinary linear detrending without adjustment to physical bounds
        ind_coefs = month_data.polyfit(dim='time', deg=1, skipna=True)
        ind_trend = yrs_ind * ind_coefs.sel(degree=1) + ind_coefs.sel(degree=0)

        #use the month's data with modified time coordinates to compute the detrended data
        detrended_ind = month_data - ind_trend
        detrended_ind['time'] = month_data_['time'] #now calculations have taken place revert to the original time coordinates
        detrended_ind = xr.Dataset({'SIC': detrended_ind['polyfit_coefficients']})
        ##############################################################################################
        #save to NetCDF 
        detrended_ind.attrs = {'Description': 'Detrended Arctic sea ice concentrations (SIC) the model {}. Years {}-{}, month of {}. '\
                               +'Detrended relative to the individual ensemble member linear trend.'.format(model_name, month_names[month_-1], start_yr, end_yr), 
                              'Units'      : '%',
                              'Timestamp'  : str(datetime.datetime.utcnow().strftime("%H:%M UTC %a %Y-%m-%d")),
                              'Data source': 'CLIVAR Large Ensemble Archive, doi:10.1038/s41558-020-0731-2',
                              'Analysis'   : 'https://github.com/chrisrwp/synthetic-ensemble/SIC/Detrend_different_dates_SIC_models.ipynb'}
        
        detrended_ind.to_netcdf('/glade/scratch/cwpowell/Synthetic_ensemble_revisions/SIC/Detrended/'\
                                +'{}_detrended_{}_individual_{}_{}.nc'.format(model_name, str(month_).zfill(2), start_yr, end_yr))  

# Observations

In [10]:
data_path = '/glade/campaign/univ/ucub0084/Synthetic_ensemble/'

start_yr = 1979
end_yr   = 2010

for dataset_name in ['NSIDC_CDR', 'NSIDC_NT', 'NSIDC_BT', 'HadISST1']: 
    print(datetime.datetime.now(), dataset_name)
    
    if dataset_name == 'HadISST1':
        data = xr.open_dataset(data_path+'Raw_data/observations/HadISST/HadISST1_NH_79-20_filled.nc')
        lat_lon = ['latitude', 'longitude']
        dataset_var = 'sic'
        data_doi = 'doi:10.1029/2002JD002670'
    else:
        data = xr.open_dataset(data_path+'Raw_data/observations/NSIDC_CDR_v4/SIC_CDR_BT_NT_79-20_filled.nc')
        lat_lon = ['ygrid', 'xgrid']
        dataset_var = dataset_name.split('_')[1]
        data_doi = 'doi:10.7265/efmz-2t65'
            
    ind_det = []
    for month_ in np.arange(1,13):
        
        month_data = data[dataset_var].sel(time=data['time.month']==month_).sel(time=slice(str(start_yr), str(end_yr)))
        orig_time = month_data['time']
        
        ##############################################################################################
        #generate a matrix of year values for computing the trend with the trend coefficients
        yrs_ind = np.empty((len(month_data['time']), len(month_data[lat_lon[0]]), len(month_data[lat_lon[1]])))

        for yr_i, yr in enumerate(np.arange(start_yr,end_yr+1)):
            yrs_ind[yr_i] = np.ones((len(month_data[lat_lon[0]]), len(month_data[lat_lon[1]]))) * yr

        yrs_ind = xr.DataArray(data = yrs_ind, coords = {'time':month_data['time'], lat_lon[0]:month_data[lat_lon[0]], 
                               lat_lon[1]:month_data[lat_lon[1]]}, dims = ['time', lat_lon[0], lat_lon[1]])  

        ##############################################################################################
        #calculate the linear trend coefficients and the coresponding values each year for the individual trend
        ind_coefs = month_data.polyfit(dim='time', deg=1, skipna=True)
        ind_trend = yrs_ind * ind_coefs.sel(degree=1) + ind_coefs.sel(degree=0)

        #now member data and the trends are in the same time coordinates, compute anomalies
        detrended_ind = month_data - ind_trend
        detrended_ind['time'] = orig_time #now calculations have taken place revert to the original time coordinates
    
        ind_det.append(detrended_ind)
        
    ind_det = xr.concat((ind_det),dim='time')
    
    ind_det.attrs = {'Description': 'Detrended Arctic sea ice concentrations (SIC) for the observational dataset {}. '\
        +'Years {}-{}, all months. Detrended relative to the linear trend of the each month'\
        +'.'.format(dataset_name, start_yr, end_yr), 
        'Units'      : '%',
        'Timestamp'  : str(datetime.datetime.utcnow().strftime("%H:%M UTC %a %Y-%m-%d")),
        'Data source': data_doi,
        'Analysis'   : 'https://github.com/chrisrwp/synthetic-ensemble/SIC/Detrend_different_dates_SIC_obs_models.ipynb',
        }
    
    ind_det = ind_det.rename({'polyfit_coefficients':'SIC'})
    ind_det.to_netcdf('/glade/scratch/cwpowell/Synthetic_ensemble_revisions/SIC/Detrended/'\
                      +'{}_detrended_individual_{}_{}.nc'.format(dataset_name, start_yr, end_yr))